In [1]:
import requests
import time
import json
import csv
from bs4 import BeautifulSoup

In [1]:
import mysql.connector
import random
from faker import Faker
from snowflake import SnowflakeGenerator, Snowflake
from datetime import datetime


db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

In [3]:
connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

user_id = 1450492617670998016

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
}

In [5]:
url1 = "https://api-crownx.winmart.vn/mt/api/web/v1/category"

In [6]:
fake = Faker('vi_VN')
sf = Snowflake.parse(856165981072306191, 1288834974657)
gen = SnowflakeGenerator.from_snowflake(sf)

In [10]:
# tao bang category
categoryNames = []

def extract_category(cat, parent_code = None):
    parent = cat.get("parent", {})
    
    description = parent.get("description")
    clean_description = BeautifulSoup(description, "html.parser").get_text()

    codeParent = parent_code if parent_code is not None else next(gen)
    category_info = {
        "id": next(gen),
        "name": parent.get("name"),
        "description": clean_description,
        "seo_name": parent.get("seoName"),
        "parentId": codeParent,   # lưu parentId theo code_parent
        "small_image": parent.get("imageUrl"),
        "codeBrand": next(gen),
        "level": parent.get("level")
    }

    # check trùng name
    if not any(c["name"] == category_info["name"] for c in categoryNames):
        categoryNames.append(category_info)

        # duyệt con
        for child in cat.get("lstChild", []):
            extract_category(child, parent_code=category_info["parentId"])


# gọi API
res1 = requests.get(url1, headers=headers)
res1.raise_for_status()
categoryData = res1.json().get("data", [])

for cat in categoryData:
    extract_category(cat, cat.get("parent", {}).get("codeParent"))

print(f"Số danh mục lấy được: {len(categoryNames)}")
for c in categoryNames[:10]:
    print(c)


Số danh mục lấy được: 92
{'id': 1970993987625332737, 'name': 'Giá Siêu Rẻ', 'description': '', 'seo_name': 'gia-sieu-re--c114', 'parentId': 1970993987625332736, 'small_image': 'https://s3-hcmc02.higiocloud.vn/images/2025/02/group-10_gia-sieu-re_6-20250211153756.png', 'codeBrand': 1970993987625332738, 'level': 1}
{'id': 1970993987625332740, 'name': 'Ưu Đãi Hội Viên', 'description': '', 'seo_name': 'uu-dai-hoi-vien--c51', 'parentId': 1970993987625332739, 'small_image': 'https://s3-hcmc02.higiocloud.vn/images/2023/06/artboard-14-20230608080819.gif', 'codeBrand': 1970993987625332741, 'level': 1}
{'id': 1970993987625332743, 'name': 'Sữa các loại', 'description': 'Sữa - Sản Phẩm Từ Sữa', 'seo_name': 'sua-cac-loai--c08', 'parentId': 1970993987625332742, 'small_image': 'https://s3-hcmc02.higiocloud.vn/images/2025/02/sua-20250211153053.png', 'codeBrand': 1970993987625332744, 'level': 1}
{'id': 1970993987625332745, 'name': 'Sữa Tươi', 'description': 'Sữa Tươi', 'seo_name': 'sua-tuoi--c0133', 'pa

In [11]:
def generate_meta_keywords(name: str):
    name_lower = name.lower()
    keywords = [name]

    if "sữa" in name_lower:
        keywords.extend(["sữa tươi", "sữa bột", "sữa hạt", "sữa đặc"])
    elif "trái cây" in name_lower or "hoa quả" in name_lower:
        keywords.extend(["trái cây tươi", "hoa quả", "trái cây nhập khẩu"])
    elif "bia" in name_lower:
        keywords.extend(["bia lon", "bia chai", "bia nhập khẩu"])
    elif "nước ngọt" in name_lower:
        keywords.extend(["nước giải khát", "nước có gas", "nước suối"])
    else:
        # fallback: lấy thêm vài biến thể từ name
        keywords.extend([f"{name} giá rẻ", f"mua {name} online"])

    return ", ".join(set(keywords))  # loại bỏ trùng


In [262]:
for category in categoryNames:
    id = category["id"]
    name = category["name"]
    seo_title = category["seo_name"] or fake.sentence(nb_words=7)
    description = category["description"] or fake.sentence(nb_words=20)
    status = 1
    parent_id = category["parentId"]
    meta_keyword = generate_meta_keywords(name)
    create_by = user_id
    update_by = user_id
    create_at = fake.date_time_between(start_date="-2y", end_date="now").strftime("%Y-%m-%d %H:%M:%S")
    update_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    small_image = category["small_image"]
    code_brand = category["codeBrand"]
    level = category["level"]
    
    sql = """
    INSERT INTO category
    (id, name, seo_title, description, status, parent_id, meta_keyword, create_by, create_at, update_by, update_at, small_image, code_brand, level)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.execute(sql, (id, name, seo_title, description, status, parent_id, meta_keyword, create_by, create_at, update_by, update_at, small_image, code_brand, level))
    
connect_mysql.commit()
cursor.close()
connect_mysql.close()

print("Da insert data vào table category")


Da insert data vào table category


Chỗ này là bắt đầu tạo brand

In [12]:
# Tao bangg brand

brandAll = []

def extract_brand(cat):
    parent = cat.get("parent", {}).get("brands", [])
    catName = cat.get("parent", {}).get("name")
    
    catId = None
    for c in categoryNames:
        if c["name"] == catName:
            catId = c["codeBrand"]
            break

    for p in parent:
        brand_info = {
            "id": next(gen),
            "name": p.get("brandName"),
            "image": p.get("imgUrl"),
            "catId": catId
        }

        # check trùng name
        if not any(c["name"] == brand_info["name"] for c in brandAll):
            brandAll.append(brand_info)

        # duyệt con
    for child in cat.get("lstChild", []):
            extract_brand(child)


# gọi API
res1 = requests.get(url1, headers=headers)
res1.raise_for_status()
brandData = res1.json().get("data", [])

for brand in brandData:
    extract_brand(brand)

print(f"Số danh mục lấy được: {len(brandAll)}")
for c in brandAll[:10]:
    print(c)


Số danh mục lấy được: 138
{'id': 1970994050657333248, 'name': 'ANCHOR', 'image': 'https://cdn-crownx.winmart.vn/images/prod/anchor.png', 'catId': 1970993987625332744}
{'id': 1970994050657333249, 'name': 'ĐÀ LẠT MILK', 'image': 'https://cdn-crownx.winmart.vn/images/prod/dalatmilk.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333250, 'name': 'DUTCH LADY', 'image': 'https://cdn-crownx.winmart.vn/images/prod/dutch-lady1.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333251, 'name': 'LOTHAMILK', 'image': 'https://cdn-crownx.winmart.vn/images/prod/lothamilk.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333252, 'name': 'MEADOW FRESH', 'image': 'https://cdn-crownx.winmart.vn/images/prod/meadow-fresh.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333253, 'name': 'MEIJI', 'image': 'https://cdn-crownx.winmart.vn/images/prod/meiji.png', 'catId': 1970993987625332744}
{'id': 1970994050657333254, 'name': 'NESTLE', 'image': 'https://cdn-crownx.winmart.vn/images/pr

In [13]:
url = "https://api-crownx.winmart.vn/it/api/web/v3/item/category"

categories = [
    {"slug": "rau-cu-trai-cay--c02"},
    {"slug": "sua-cac-loai--c08"},
    {"slug": "hoa-pham-tay-rua--c10"},
    {"slug": "cham-soc-ca-nhan--c11"},
    {"slug": "thit-hai-san-tuoi--c03"},
    {"slug": "banh-keo--c07"},
    {"slug": "do-uong-co-con--c31"},
    {"slug": "do-uong-giai-khat--c09"},
    {"slug": "mi-thuc-pham-an-lien--c34"},
    {"slug": "thuc-pham-kho--c06"},
    {"slug": "thuc-pham-che-bien--c04"},
    {"slug": "gia-vi--c35"},
    {"slug": "thuc-pham-dong-lanh--c05"},
    {"slug": "trung-dau-hu--c33"},
    {"slug": "cham-soc-be--c12"},
    {"slug": "do-dung-gia-dinh--c25"},
    {"slug": "van-phong-pham-do-choi--c27"},
    {"slug": "dien-gia-dung--c26"},
]

all_products = []

In [14]:
def extract_brand_pro(cat_data):
    items = cat_data.get("items", [])
    category_name = cat_data.get("categoryName", "")
    for item in items:
        
        catId = None;
        brand_name = item.get("brandName")
        
        for c in categoryNames:
            if c["name"] == item.get("categoryName"):
                catId = c["codeBrand"]
                break
        
        brand_info = {
            "id": next(gen),
            "name": brand_name,
            "image": item.get("mediaUrl") or "",
            "catId": catId
        }
        if not any(b["name"] == brand_info["name"] for b in brandAll):
            brandAll.append(brand_info)


In [15]:
for cat in categories:
    slug = cat["slug"]
    page = 1
    while True:
        params = {
            "orderByDesc": "true",
            "pageNumber": page,
            "pageSize": 38,
            "slug": slug,
            "storeCode": 1535,
            "storeGroupCode": 1998,
        }
        
        res = requests.get(url, params=params, headers=headers)
        res.raise_for_status()
        data = res.json()   # <-- đổi tên biến

        items = data.get("data", {}).get("items", [])   # <-- phải là "data" chứ không phải "date"
        paging = data.get("paging", {})
        

        # Gọi hàm extract_brand_pro và truyền toàn bộ data
        extract_brand_pro(data.get("data", {}))
        

        if page >= paging.get("totalPages", 0):
                break
            
        page += 1
        time.sleep(0.2)  # tránh spam server quá nhanh
        print(f"Số danh mục lấy được: {len(brandAll)}")
        for c in brandAll[:10]:
            print(c)


Số danh mục lấy được: 145
{'id': 1970994050657333248, 'name': 'ANCHOR', 'image': 'https://cdn-crownx.winmart.vn/images/prod/anchor.png', 'catId': 1970993987625332744}
{'id': 1970994050657333249, 'name': 'ĐÀ LẠT MILK', 'image': 'https://cdn-crownx.winmart.vn/images/prod/dalatmilk.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333250, 'name': 'DUTCH LADY', 'image': 'https://cdn-crownx.winmart.vn/images/prod/dutch-lady1.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333251, 'name': 'LOTHAMILK', 'image': 'https://cdn-crownx.winmart.vn/images/prod/lothamilk.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333252, 'name': 'MEADOW FRESH', 'image': 'https://cdn-crownx.winmart.vn/images/prod/meadow-fresh.jpg', 'catId': 1970993987625332744}
{'id': 1970994050657333253, 'name': 'MEIJI', 'image': 'https://cdn-crownx.winmart.vn/images/prod/meiji.png', 'catId': 1970993987625332744}
{'id': 1970994050657333254, 'name': 'NESTLE', 'image': 'https://cdn-crownx.winmart.vn/images/pr

Số danh mục lấy được: 92
{'id': 1970987797025566721, 'name': 'Giá Siêu Rẻ', 'description': '', 'seo_name': 'gia-sieu-re--c114', 'parentId': 1970987797025566720, 'small_image': 'https://s3-hcmc02.higiocloud.vn/images/2025/02/group-10_gia-sieu-re_6-20250211153756.png', 'codeBrand': 1970987797025566722, 'level': 1}
{'id': 1970987797025566724, 'name': 'Ưu Đãi Hội Viên', 'description': '', 'seo_name': 'uu-dai-hoi-vien--c51', 'parentId': 1970987797025566723, 'small_image': 'https://s3-hcmc02.higiocloud.vn/images/2023/06/artboard-14-20230608080819.gif', 'codeBrand': 1970987797025566725, 'level': 1}
{'id': 1970987797025566727, 'name': 'Sữa các loại', 'description': 'Sữa - Sản Phẩm Từ Sữa', 'seo_name': 'sua-cac-loai--c08', 'parentId': 1970987797025566726, 'small_image': 'https://s3-hcmc02.higiocloud.vn/images/2025/02/sua-20250211153053.png', 'codeBrand': 1970987797025566728, 'level': 1}
{'id': 1970987797025566729, 'name': 'Sữa Tươi', 'description': 'Sữa Tươi', 'seo_name': 'sua-tuoi--c0133', 'pa

In [16]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

common_keywords = [
    "chất lượng cao",
    "hương vị tuyệt vời",
    "an toàn cho sức khỏe",
    "được tin dùng",
    "sản phẩm chính hãng",
    "mang lại trải nghiệm tốt",
    "giúp cải thiện cuộc sống",
    "dễ sử dụng",
    "hiệu quả vượt trội",
    "đảm bảo uy tín",
]

def generate_common_meta_keywords(n=3):
    """
    Sinh n từ khóa ngẫu nhiên từ bộ từ khóa chung
    """
    keywords = random.sample(common_keywords, k=n)
    return ", ".join(keywords)

for brand in brandAll:
    id = brand["id"]
    name = brand["name"]
    description = generate_common_meta_keywords(3) or fake.sentence(nb_words=20)
    
    country = "Việt Nam" if fake.boolean(chance_of_getting_true=60) else fake.country()
    status = 1
    created_by = user_id
    update_by = user_id
    create_at = fake.date_time_between(start_date="-2y", end_date='now').strftime('%Y-%m-%d %H:%M:%S')
    update_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cat_id = brand.get("catId")
    image = brand["image"]
    
    sql = """
    INSERT INTO brand
    (id, name, description, country, create_at, update_at, created_by, update_by, status, cat_id, image)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.execute(sql, (id, name, description, country, create_at, update_at, created_by, update_by, status, cat_id, image))
    
connect_mysql.commit()
cursor.close()
connect_mysql.close()

print("Da insert data vào table brand")


Da insert data vào table brand


Tạo product

In [18]:
# Tao product
url = "https://api-crownx.winmart.vn/it/api/web/v3/item/category"

categories = [
    {"slug": "rau-cu-trai-cay--c02"},
    {"slug": "sua-cac-loai--c08"},
    {"slug": "hoa-pham-tay-rua--c10"},
    {"slug": "cham-soc-ca-nhan--c11"},
    {"slug": "thit-hai-san-tuoi--c03"},
    {"slug": "banh-keo--c07"},
    {"slug": "do-uong-co-con--c31"},
    {"slug": "do-uong-giai-khat--c09"},
    {"slug": "mi-thuc-pham-an-lien--c34"},
    {"slug": "thuc-pham-kho--c06"},
    {"slug": "thuc-pham-che-bien--c04"},
    {"slug": "gia-vi--c35"},
    {"slug": "thuc-pham-dong-lanh--c05"},
    {"slug": "trung-dau-hu--c33"},
    {"slug": "cham-soc-be--c12"},
    {"slug": "do-dung-gia-dinh--c25"},
    {"slug": "van-phong-pham-do-choi--c27"},
    {"slug": "dien-gia-dung--c26"},
]

all_products = []

In [19]:
for cat in categories:
    slug = cat["slug"]
    page = 1
    while True:
        params = {
            "orderByDesc": "true",
            "pageNumber": page,
            "pageSize": 38,
            "slug": slug,
            "storeCode": 1535,
            "storeGroupCode": 1998,
        }
        
        res = requests.get(url, params=params, headers=headers)
        res.raise_for_status()
        data = res.json()   # <-- đổi tên biến

        items = data.get("data", {}).get("items", [])   # <-- phải là "data" chứ không phải "date"
        paging = data.get("paging", {})
        

        all_products.append(items)
        print(f"{slug} - page {page}/{paging.get('totalPages')} -> {len(items)} items")

        if page >= paging.get("totalPages", 0):
            break
        page += 1
        time.sleep(0.2)  # tránh spam server quá nhanh

print(f"Tổng số sản phẩm lấy được: {len(all_products)}")



rau-cu-trai-cay--c02 - page 1/4 -> 38 items
rau-cu-trai-cay--c02 - page 2/4 -> 38 items
rau-cu-trai-cay--c02 - page 3/4 -> 38 items
rau-cu-trai-cay--c02 - page 4/4 -> 28 items
sua-cac-loai--c08 - page 1/8 -> 38 items
sua-cac-loai--c08 - page 2/8 -> 38 items
sua-cac-loai--c08 - page 3/8 -> 38 items
sua-cac-loai--c08 - page 4/8 -> 38 items
sua-cac-loai--c08 - page 5/8 -> 38 items
sua-cac-loai--c08 - page 6/8 -> 38 items
sua-cac-loai--c08 - page 7/8 -> 38 items
sua-cac-loai--c08 - page 8/8 -> 35 items
hoa-pham-tay-rua--c10 - page 1/5 -> 38 items
hoa-pham-tay-rua--c10 - page 2/5 -> 38 items
hoa-pham-tay-rua--c10 - page 3/5 -> 38 items
hoa-pham-tay-rua--c10 - page 4/5 -> 38 items
hoa-pham-tay-rua--c10 - page 5/5 -> 6 items
cham-soc-ca-nhan--c11 - page 1/17 -> 38 items
cham-soc-ca-nhan--c11 - page 2/17 -> 38 items
cham-soc-ca-nhan--c11 - page 3/17 -> 38 items
cham-soc-ca-nhan--c11 - page 4/17 -> 38 items
cham-soc-ca-nhan--c11 - page 5/17 -> 38 items
cham-soc-ca-nhan--c11 - page 6/17 -> 38 it

In [20]:
print(all_products[0])

[{'id': '95024a59-2d2a-4bcc-9449-f19f095a31cf', 'itemNo': '10602355', 'brand': '10023', 'seoName': 'kiwi-xanh-newzealand-2traihop--s10602355', 'isAlcohol': False, 'mediaUrl': 'https://s3-hcmc02.higiocloud.vn/images/2022/162428311443110602355-hop-sua-rua-tay-kleen-huong-dào-chai-500ml-og.jpg', 'description': 'Kiwi xanh Newzealand (2Trái/Hộp)', 'shortDescription': 'Kiwi xanh Newzealand (2Trái/Hộp)', 'name': 'Kiwi xanh Newzealand (2Trái/Hộp)', 'itemType': 'ZTRD', 'uomId': '36672f69-e64c-4690-9e6c-671bca30377f', 'uom': 'HOP', 'uomName': 'Hộp', 'quantityPerUnit': 1.0, 'sku': '10602355HOP', 'barcode': '8936052001417', 'price': 55000.0, 'salePrice': 39000.0, 'quantity': 24.0, 'promotionCode': '2300769518', 'promotionType': 'ZB10', 'scaleType': 'EQUAL', 'scaleQuantity': 1.0, 'mch1': '1', 'mch1Name': 'Thực phẩm', 'mch2': '101', 'mch2Name': 'Thực phẩm Tươi sống, Chế biến', 'mch3': '10106', 'mch3Name': 'Trái cây', 'mch4': '1010602', 'mch4Name': 'Trái cây nhập khẩu', 'mch5': '101060205', 'mch5Name

In [21]:
def get_id(table, name, key, conn):
    with conn.cursor() as cursor:  # dùng context manager để tự động đóng cursor
        sql = f"SELECT {key} FROM {table} WHERE name = %s LIMIT 1"
        cursor.execute(sql, (name,))
        row = cursor.fetchone()
        return row[0] if row else None

In [22]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

def get_data(table, cursor):
    sql = f"select name from {table}"
    cursor.execute(sql)
    rows = cursor.fetchall()
    return [row[0] for row in rows]
    
categories = get_data("category", cursor)
brands = get_data("brand", cursor)


for page_items in all_products:  # duyệt từng page
    for product in page_items:    # product là dict thật
        id = next(gen)
        name = product.get("name", "")
        
        brand = product["brandName"]
        brand_id = get_id("brand", brand, "id", connect_mysql)
        
        category = product["categoryName"]
        category_id = get_id("category", category, "id", connect_mysql)
        
        sku = product["sku"]
        seo_title = product["seoName"] or fake.sentence(nb_words=7)
        description = product["description"] or fake.sentence(nb_words=20)
        status = 1
        
        tag = product["mch3Name"]
        image = product.get("mediaUrl", "") or fake.image_url()
        list_image = None
        vat = round(random.uniform(0, 0.2), 3)
        price_sell = product["price"]
        price_normal = price_sell - round(random.uniform(10000, 20000), 3) + vat * price_sell
        promotion_price = product["salePrice"]
        
        weight = round(random.uniform(0.1, 5), 3)
        warranty = f"{random.randint(1,12)} tháng"
        hot = None
        view_count = random.randint(0, 100000)
        
        meta_keyword = ", ".join([
            product.get("uomName", ""),
            product.get("mch1Name", ""),
            product.get("mch5Name", "")
        ])
        
        create_by = user_id
        create_at = fake.date_time_between(start_date="-2y", end_date='now').strftime('%Y-%m-%d %H:%M:%S')
        update_by = user_id
        update_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        sellable = 1

        sql = """
        INSERT INTO product (
            id, sku, name, seo_title, description, status, tag, image, list_image, price_normal, price_sell, promotion_price, vat, weight, warranty,
            hot, view_count, category_id, brand_id, meta_keyword, create_by, create_at, update_by, update_at, sellable
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (
            id, sku, name, seo_title, description, status, tag, image, list_image, price_normal, price_sell, promotion_price, vat, weight, warranty,
            hot, view_count, category_id, brand_id, meta_keyword, create_by, create_at, update_by, update_at, sellable
        ))

connect_mysql.commit()
cursor.close()
connect_mysql.close()

print("Đã insert data vào table product")


Đã insert data vào table product
